In [119]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np
import string

In [120]:
f=pd.read_csv("combined_final_orginal_dataset.csv")

C:\Users\mksuj\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3106: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


#Remove unneccasry columns and create a new csv

In [121]:
keep_col = ['created_at','lang','text']
new_f = f[keep_col]
new_f.to_csv("Updated_Orginal.csv", index=False)

#Remove non english

In [122]:
new_f=pd.read_csv("Updated_Orginal.csv")
new_f = new_f[new_f["lang"].str.contains("en") == True]
new_f.to_csv("Updated_Orginal_Eng.csv", index=False)

In [123]:
num_rows = new_f.shape[0]
print(num_rows)

161563


#remove the duplicate rows based on the 'text' column

In [124]:
new_f=pd.read_csv("Updated_Orginal_Eng.csv")

# Remove duplicate rows based on the 'text' column
new_f = new_f.drop_duplicates(subset =['created_at','lang','text'], keep='first', inplace=False, ignore_index=False)

new_f.to_csv('Updated_Distinct_Orginal.csv', index=False)


In [125]:
num_rows = new_f.shape[0]
print(num_rows)

142541


Data Preprocessing : Data Cleaning

In [126]:
#Remove http web references

In [127]:
new_f['Content_Parsed_0'] = new_f['text'].str.replace('https:\/\/.*|http:\/\/.*',"")

C:\Users\mksuj\AppData\Local\Temp\ipykernel_18552\740329011.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  new_f['Content_Parsed_0'] = new_f['text'].str.replace('https:\/\/.*|http:\/\/.*',"")


In [128]:
#Remove emails @

In [129]:
new_f['Content_Parsed_1'] = new_f['Content_Parsed_0'].str.replace(r"\S*@\S*\s?", "")

C:\Users\mksuj\AppData\Local\Temp\ipykernel_18552\2517235319.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  new_f['Content_Parsed_1'] = new_f['Content_Parsed_0'].str.replace(r"\S*@\S*\s?", "")


In [130]:
#Remove possessive pronouns

In [131]:
new_f['Content_Parsed_2'] = new_f['Content_Parsed_1'].str.replace("'s", "")

In [132]:
#Remove special characters

In [133]:
new_f['Content_Parsed_3'] = new_f['Content_Parsed_2'].str.replace("\r", "")
new_f['Content_Parsed_3'] = new_f['Content_Parsed_3'].str.replace("\n ", "")
new_f['Content_Parsed_3'] = new_f['Content_Parsed_3'].str.replace("\n", "")
new_f['Content_Parsed_3'] = new_f['Content_Parsed_3'].str.replace("    ", "")

In [134]:
new_f['Content_Parsed_4'] = new_f['Content_Parsed_3'].str.replace("&", "")
new_f['Content_Parsed_4'] = new_f['Content_Parsed_4'].str.replace("#", "")

In [135]:
#Lowercasing the text

In [195]:
new_f['Content_Parsed_5'] = new_f['Content_Parsed_4'].str.lower()

In [196]:
#Remove Punctuation signs

In [206]:
#.replace({"$^'|'": ""}, regex=True)
punctuation_signs = list("!#$%&'""()*+,-./:;<=>?@[\]^_`{|}~")
new_f['Content_Parsed_6'] = new_f['Content_Parsed_5']

for punct_sign in punctuation_signs:
    new_f['Content_Parsed_6'] = new_f['Content_Parsed_6'].str.replace(punct_sign, " ")
    
new_f['Content_Parsed_6'] = new_f['Content_Parsed_6'].str.replace('"', '')
new_f['Content_Parsed_6'] = new_f['Content_Parsed_6'].replace({"$^'|'": ""}, regex=True)
new_f['Content_Parsed_6'] = new_f['Content_Parsed_6'].str.replace('’','')
new_f['Content_Parsed_6'] = new_f['Content_Parsed_6'].str.replace('“','')
new_f['Content_Parsed_6'] = new_f['Content_Parsed_6'].str.replace('”','')

C:\Users\mksuj\AppData\Local\Temp\ipykernel_18552\489861548.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  new_f['Content_Parsed_6'] = new_f['Content_Parsed_6'].str.replace(punct_sign, " ")


In [207]:
#Remove numbers

In [208]:
new_f['Content_Parsed_7'] = new_f['Content_Parsed_6'].str.replace("\d+", "")

C:\Users\mksuj\AppData\Local\Temp\ipykernel_18552\1142136919.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  new_f['Content_Parsed_7'] = new_f['Content_Parsed_6'].str.replace("\d+", "")


In [209]:
#Remove all single characters

In [210]:
new_f['Content_Parsed_8'] = new_f['Content_Parsed_7'].str.replace(r'\b\w\b','').str.replace(r'\s+', ' ')

C:\Users\mksuj\AppData\Local\Temp\ipykernel_18552\699451120.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  new_f['Content_Parsed_8'] = new_f['Content_Parsed_7'].str.replace(r'\b\w\b','').str.replace(r'\s+', ' ')


In [211]:
#Remove whitespaces

In [212]:
new_f['Content_Parsed_9'] = new_f['Content_Parsed_8'].str.replace(r"^\s+","")
new_f['Content_Parsed_9'] = new_f['Content_Parsed_8'].str.replace(r"\s+"," ")

C:\Users\mksuj\AppData\Local\Temp\ipykernel_18552\1516970677.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  new_f['Content_Parsed_9'] = new_f['Content_Parsed_8'].str.replace(r"^\s+","")
C:\Users\mksuj\AppData\Local\Temp\ipykernel_18552\1516970677.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  new_f['Content_Parsed_9'] = new_f['Content_Parsed_8'].str.replace(r"\s+"," ")


In [213]:
new_f.to_csv('pre_processed_data.csv')

In [148]:
#orginal text
new_f.loc[1]['text']

"We all need to fight #COVID19.\nAnyone can get infected and spread the disease to others.\nSome young, healthy people have become seriously sick, and some have even died.\n\nLow risk isn't no risk. https://t.co/585OnotaMS"

In [149]:
#remove http
new_f.loc[1]['Content_Parsed_0']

"We all need to fight #COVID19.\nAnyone can get infected and spread the disease to others.\nSome young, healthy people have become seriously sick, and some have even died.\n\nLow risk isn't no risk. "

In [150]:
#remove email @
new_f.loc[1]['Content_Parsed_1']

"We all need to fight #COVID19.\nAnyone can get infected and spread the disease to others.\nSome young, healthy people have become seriously sick, and some have even died.\n\nLow risk isn't no risk. "

In [151]:
#remove posessive pronoun
new_f.loc[1]['Content_Parsed_2']

"We all need to fight #COVID19.\nAnyone can get infected and spread the disease to others.\nSome young, healthy people have become seriously sick, and some have even died.\n\nLow risk isn't no risk. "

In [152]:
#remove special characters
new_f.loc[1]['Content_Parsed_3']

"We all need to fight #COVID19.Anyone can get infected and spread the disease to others.Some young, healthy people have become seriously sick, and some have even died.Low risk isn't no risk. "

In [153]:
#remove special characters
new_f.loc[1]['Content_Parsed_4']

"We all need to fight COVID19.Anyone can get infected and spread the disease to others.Some young, healthy people have become seriously sick, and some have even died.Low risk isn't no risk. "

In [154]:
#lowercase
new_f.loc[1]['Content_Parsed_5']

"we all need to fight covid19.anyone can get infected and spread the disease to others.some young, healthy people have become seriously sick, and some have even died.low risk isn't no risk. "

In [155]:
#remove punctuation signs
new_f.loc[1]['Content_Parsed_6']

'we all need to fight covid19 anyone can get infected and spread the disease to others some young  healthy people have become seriously sick  and some have even died low risk isn t no risk  '

In [156]:
#Remove numbers
new_f.loc[1]['Content_Parsed_7']

'we all need to fight covid anyone can get infected and spread the disease to others some young  healthy people have become seriously sick  and some have even died low risk isn t no risk  '

In [193]:
# remove all single characters 
new_f.loc[1]['Content_Parsed_8']

'we all need to fight covid anyone can get infected and spread the disease to others some young healthy people have become seriously sick and some have even died low risk isn no risk '

In [194]:
#remove whitespaces
new_f.loc[1]['Content_Parsed_9']

'we all need to fight covid anyone can get infected and spread the disease to others some young healthy people have become seriously sick and some have even died low risk isn no risk '

In [171]:
new_f.tail()

,created_at,lang,text,Content_Parsed_0,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6,Content_Parsed_7,Content_Parsed_8,Content_Parsed_9
161558,Mon Jan 24 00:59:53 +0000 2022,en,@MichaelFieldNZ Maybe a fav of the older gener...,@MichaelFieldNZ Maybe a fav of the older gener...,Maybe a fav of the older generation perhaps (o...,Maybe a fav of the older generation perhaps (o...,Maybe a fav of the older generation perhaps (o...,Maybe a fav of the older generation perhaps (o...,maybe a fav of the older generation perhaps (o...,maybe a fav of the older generation perhaps o...,maybe a fav of the older generation perhaps o...,maybe fav of the older generation perhaps our ...,maybe fav of the older generation perhaps our ...
161559,Mon Jan 24 00:59:54 +0000 2022,en,🚨Very Bad Idea!🚨\nPutting hospitalized #Covid1...,🚨Very Bad Idea!🚨\nPutting hospitalized #Covid1...,🚨Very Bad Idea!🚨\nPutting hospitalized #Covid1...,🚨Very Bad Idea!🚨\nPutting hospitalized #Covid1...,🚨Very Bad Idea!🚨Putting hospitalized #Covid19 ...,🚨Very Bad Idea!🚨Putting hospitalized Covid19 p...,🚨very bad idea!🚨putting hospitalized covid19 p...,🚨very bad idea 🚨putting hospitalized covid19 p...,🚨very bad idea 🚨putting hospitalized covid pos...,🚨very bad idea 🚨putting hospitalized covid pos...,🚨very bad idea 🚨putting hospitalized covid pos...
161560,Mon Jan 24 00:59:54 +0000 2022,en,The truckers do realize that even if the vacci...,The truckers do realize that even if the vacci...,The truckers do realize that even if the vacci...,The truckers do realize that even if the vacci...,The truckers do realize that even if the vacci...,The truckers do realize that even if the vacci...,the truckers do realize that even if the vacci...,the truckers do realize that even if the vacci...,the truckers do realize that even if the vacci...,the truckers do realize that even if the vacci...,the truckers do realize that even if the vacci...
161561,Mon Jan 24 00:59:53 +0000 2022,en,@oneunderscore__ Exploiting of the tragedy of ...,@oneunderscore__ Exploiting of the tragedy of ...,Exploiting of the tragedy of people who suffer...,Exploiting of the tragedy of people who suffer...,Exploiting of the tragedy of people who suffer...,Exploiting of the tragedy of people who suffer...,exploiting of the tragedy of people who suffer...,exploiting of the tragedy of people who suffer...,exploiting of the tragedy of people who suffer...,exploiting of the tragedy of people who suffer...,exploiting of the tragedy of people who suffer...
161562,Mon Jan 24 00:59:54 +0000 2022,en,"K'taka reported 50k mark covid cases, with 50,...","K'taka reported 50k mark covid cases, with 50,...","K'taka reported 50k mark covid cases, with 50,...","K'taka reported 50k mark covid cases, with 50,...","K'taka reported 50k mark covid cases, with 50,...","K'taka reported 50k mark covid cases, with 50,...","k'taka reported 50k mark covid cases, with 50,...",k taka reported 50k mark covid cases with 50 ...,k taka reported k mark covid cases with in ...,taka reported mark covid cases with in number...,k taka reported mark covid cases with in numbe...
